In [10]:
cd BiS495/GNN

[Errno 2] No such file or directory: 'BiS495/GNN'
/work/home/nhkim/BiS495/GNN


In [11]:
import os
import logging
import numpy as np
import pandas as pd
import random
import adabound
import pickle
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

import torch

# Env
from utils import *
from model_GAT import *
from options import parse_args
from test_model import test

In [12]:
opt = parse_args()

----------------- Options ---------------
                 act_type: none                          
               adj_thresh: 0.05                          
                    alpha: 0.2                           
               batch_size: 32                            
                  cnv_dim: 0                             
                  dropout: 0.2                           
                 final_lr: 0.1                           
                  gpu_ids: 3,4,5                         
                input_dim: 1                             
                label_dim: 2                             
               lambda_cox: 1                             
               lambda_nll: 1                             
               lambda_reg: 0.0003                        
            lin_input_dim: 958                           
                       lr: 0.0001                        
                lr_policy: linear                        
                model_dir: ./p

In [13]:
rf_avg = 0
svm_avg = 0
lr_avg = 0
svm_auroc = 0
lr_auroc = 0
rf_auroc = 0

In [14]:
def data(k):
    tr_features, tr_labels, te_features, te_labels, adj_matrix = load_csv_data(k, opt)
    tr_features = tr_features.detach().cpu().numpy()
    tr_features = np.squeeze(tr_features)
    tr_labels = tr_labels.detach().cpu().numpy()
    te_features = te_features.detach().cpu().numpy()
    te_features = np.squeeze(te_features)
    te_labels = te_labels.detach().cpu().numpy()

    return tr_features, tr_labels, te_features, te_labels

In [15]:
def lr_run(tr_features, tr_labels, te_features, te_labels):
    global lr_avg
    global lr_auroc

    model = LinearRegression()
    model.fit(tr_features, tr_labels)

    te_pred = model.predict(te_features)

    te_pred_binary = [1 if pred >= 0.5 else 0 for pred in te_pred]

    accuracy = accuracy_score(te_labels, te_pred_binary)
    auroc = roc_auc_score(te_labels, te_pred_binary)
    lr_avg = accuracy + lr_avg
    lr_auroc = lr_auroc + auroc

    print(f"Accuracy: {accuracy:.4f}")

In [16]:
def rf_run(tr_features, tr_labels, te_features, te_labels):
    global rf_avg
    global rf_auroc
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    model.fit(tr_features, tr_labels)

    te_pred = model.predict(te_features)

    accuracy = accuracy_score(te_labels, te_pred)
    rf_avg = accuracy + rf_avg
    auroc = roc_auc_score(te_labels, te_pred)
    rf_auroc = auroc + rf_auroc
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Auroc: {auroc:.4f}")

In [17]:
def svm_run(tr_features, tr_labels, te_features, te_labels):
    global svm_avg
    global svm_auroc

    model = svm.SVC(kernel='rbf', C=1.0, random_state=42)

    model.fit(tr_features, tr_labels)

    te_pred = model.predict(te_features)

    accuracy = accuracy_score(te_labels, te_pred)
    svm_avg = accuracy + svm_avg
    auroc = roc_auc_score(te_labels, te_pred)
    svm_auroc = auroc + svm_auroc
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Auroc: {auroc:.4f}")

In [18]:
for j in range(5):
    for i in range(1, 6):
        tr_features, tr_labels, te_features, te_labels = data(i)
        lr_run(tr_features, tr_labels, te_features, te_labels)
        rf_run(tr_features, tr_labels, te_features, te_labels)
        svm_run(tr_features, tr_labels, te_features, te_labels)

print(f"Accuracy of LR: {lr_avg/25.0:.4f}")
print(f"Accuracy of RF: {rf_avg/25.0:.4f}")
print(f"Accuracy of SVM: {svm_avg/25.0:.4f}")
print(f"Auroc of LR: {lr_auroc/25.0:.4f}")
print(f"Auroc of RF: {rf_auroc/25.0:.4f}")
print(f"Auroc of SVM: {svm_auroc/25.0:.4f}")

Loading data from: /work/home/nhkim/BiS495/GNN/input_features_labels/split1
Training features and labels: torch.Size([576, 479, 1]) torch.Size([576, 1])
Testing features and labels: torch.Size([145, 479, 1]) torch.Size([145, 1])
Adjacency matrix: torch.Size([479, 479])
Number of edges: tensor(43949)
Accuracy: 0.6207


/work/home/nhkim/anaconda3/envs/bis332-conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


Accuracy: 0.6552
Auroc: 0.6444
Accuracy: 0.6414
Auroc: 0.6404
Loading data from: /work/home/nhkim/BiS495/GNN/input_features_labels/split2


/work/home/nhkim/anaconda3/envs/bis332-conda/lib/python3.6/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Training features and labels: torch.Size([576, 479, 1]) torch.Size([576, 1])
Testing features and labels: torch.Size([145, 479, 1]) torch.Size([145, 1])
Adjacency matrix: torch.Size([479, 479])
Number of edges: tensor(41421)
Accuracy: 0.5448


...

/work/home/nhkim/anaconda3/envs/bis332-conda/lib/python3.6/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Training features and labels: torch.Size([576, 479, 1]) torch.Size([576, 1])
Testing features and labels: torch.Size([145, 479, 1]) torch.Size([145, 1])
Adjacency matrix: torch.Size([479, 479])
Number of edges: tensor(43895)
Accuracy: 0.5448


/work/home/nhkim/anaconda3/envs/bis332-conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


Accuracy: 0.6828
Auroc: 0.6777
Accuracy: 0.6483
Auroc: 0.6473
Accuracy of LR: 0.5421
Accuracy of RF: 0.6979
Accuracy of SVM: 0.6759
Auroc of LR: 0.5000
Auroc of RF: 0.6951
Auroc of SVM: 0.6743


/work/home/nhkim/anaconda3/envs/bis332-conda/lib/python3.6/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
